Ce notebook a constitué une première manipulation des données. Il est inspiré du notebook https://www.kaggle.com/rhodiumbeng/classifying-multi-label-comments-0-9741-lb

In [1]:
import configparser
import pandas as pd
import re
# import and instantiate TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
# import and instantiate the Logistic Regression model
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

configparser permet d'utiliser un fichier de configuraiton (config.cfg) pour le projet. Cela évite de mettre en ligne sur le git des informations sensibles comme l'architecture des répertoires de la machine de travail.
Un template config.dist est disponible sur le git pour définir les clés de configuration à utiliser.

# Préparation des données

In [2]:
config = configparser.ConfigParser()
config.read('../config.cfg')
# Chargement des données sous forme de dataframes pandas
train_df = pd.read_csv(config['FILES']['TRAIN'])
test_df = pd.read_csv(config['FILES']['TEST'])

print('Dimension du jeu test')
print(test_df.shape)

print('\nEchantillon des identifiants du jeu test')
print(test_df['id'][0:5])

# Chargement des labels du jeu test, donnés ultérieurement à la compétition 
test_label = pd.read_csv(config['FILES']['LABEL'])
print('\nDimension du fichier label du jeu test')
print(test_label.shape)
print('\nEchantillon des identifiants dont les labels sont disponibles dans le jeu test')
print(test_label['id'][0:5])

Dimension du jeu test
(153164, 2)

Echantillon des identifiants du jeu test
0    00001cee341fdb12
1    0000247867823ef7
2    00013b17ad220c46
3    00017563c3f7919a
4    00017695ad8997eb
Name: id, dtype: object

Dimension du fichier label du jeu test
(153164, 7)

Echantillon des identifiants dont les labels sont disponibles dans le jeu test
0    00001cee341fdb12
1    0000247867823ef7
2    00013b17ad220c46
3    00017563c3f7919a
4    00017695ad8997eb
Name: id, dtype: object


In [3]:
# Définition des différents labels disponibles sous forme de liste
cols_name = ['obscene', 'insult', 'toxic', 'severe_toxic', 'identity_hate', 'threat']

In [4]:
def clean_text(text: str)->list:
    """
    Fonction de nettoyage du texte. 
    1. Le texte est réduit à des minuscules.
    2. Les contractions sont remplacées par le mot complet.
    3. Le texte est découpé en liste en utlisant l'espace comme séparateur
    
    Args: 
        text, le texte à modifier
    
    Returns:
        text, texte modifié
    """
    text = text.lower()
    for change in [(r"what's", "what is "), (r"\'s", " "), (r"\'ve", " have "), (r"can't", "cannot "), 
                   (r"n't", " not "), (r"i'm", "i am "), (r"\'re", " are "), (r"\'d", " would "), (r"\'ll", " will "), (r"\'scuse", " excuse ")]:
        text = re.sub(change[0], change[1], text)
    text = text.strip(' ')
    return text

In [5]:
# Application de la fonction de nettoyage au jeu d'entrainement et test
train_df['comment_text'] = train_df['comment_text'].map(lambda com: clean_text(com))
test_df['comment_text'] = test_df['comment_text'].map(lambda com: clean_text(com))

La documentation de TfidfVectorizer est disponible sur https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html

In [6]:
x_train = train_df.comment_text
x_test = test_df.comment_text

# Conversion des données brutes en matrice TF-IDF
# max_features: ne considére que les 5000 mots avec le TF-IDF le plus fort
# stop_words: ne considère pas les mots implémentés dans l'option 'english' 
# (e.g petits mots très fréquents comme le, la, les)
vect = TfidfVectorizer(max_features=5000, stop_words='english')

# Apprentissage du vocabulaire présent dans le jeu d'entrainement, création d'une matrice document-terme
# 1 ligne =  1 commentaire
# 1 colonne = un des 5000 mots les plus fréquents dans le corpus
x_train_tfid = vect.fit_transform(x_train)

# Création d'une matrice document-terme à partir des commentaires du jeu test préalablement entrainé
# les 5000 mots considérés sont ceux du jeu d'entrainement
x_test_tfid = vect.transform(x_test)

In [7]:
print('Dimension de la matrice document-terme d\'entrainement')
print(x_train_tfid.shape)
print('\nDimension de la matrice document-terme test')
print(x_test_tfid.shape)

Dimension de la matrice document-terme d'entrainement
(159571, 5000)

Dimension de la matrice document-terme test
(153164, 5000)


# Classification des commentaires

Ici, on choisit de traiter chaque label de façon indépendante. On passe d'une classification multi-label à une classification multi-classe.
Pour cela, on utilise la régression logistique (voir doc: https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html)

In [8]:
logreg = LogisticRegression(C=12.0)
# C est l'inverse du facteur de régulation lambda. Ici, C est grand donc la régulation est faible. 
# Choix non justifié dans le notebook initial, nous n'avons pas fait varier ce paramètre

# Tout les commentaires n'ont pas été utilisés lors de la validation de la compétition. 
# Certains commentaires tests ont le label -1, ils faut les enlever pour le calcul de performance
test_label_strip = test_label[test_label.toxic != -1]

# Classification pour chaque label de façon indépendante
for label in cols_name:
    print('... Processing {}'.format(label))
    y = train_df[label]
    yt = test_label_strip[label]
    # on entraine le modele avec la matrice TF-IDF d'entrainement et ses labels
    logreg.fit(x_train_tfid, y)
    # on prédit ensuite les labels du jeu d'entrainement et du jeu test avec le modèle entrainé
    y_pred_X = logreg.predict(x_train_tfid)
    y_pred_t = logreg.predict(x_test_tfid)
    # On calcule l'accuracy des prédictions
    print('Training accuracy is {}'.format(accuracy_score(y, y_pred_X)))
    print('Test accuracy is {}'.format(accuracy_score(yt, y_pred_t[test_label_strip.index])))


... Processing obscene
Training accuracy is 0.9832300355327722
Test accuracy is 0.9632998843352403
... Processing insult
Training accuracy is 0.9755469352200588
Test accuracy is 0.9620338241270436
... Processing toxic
Training accuracy is 0.9639846839337975
Test accuracy is 0.9243021038481978
... Processing severe_toxic
Training accuracy is 0.9920787611784097
Test accuracy is 0.9924192691237613
... Processing identity_hate
Training accuracy is 0.9939713356436947
Test accuracy is 0.9903716902685298
... Processing threat
Training accuracy is 0.9981199591404453
Test accuracy is 0.9959204726624777


Documentation de l'accuracy:  https://scikit-learn.org/stable/modules/generated/sklearn.metrics.accuracy_score.html
Elle renvoie la fraction de labels correctement classés